Immersed in Sounds of Space by Team RIA-2 for 2023 NASA Space Apps Challenge

In [2]:
import cv2
import numpy as np
import pygame
from scipy.interpolate import interp1d

# Initialize Pygame mixer
pygame.mixer.init()

# Load audio files for different instruments
instrument_sounds = {
    'red': pygame.mixer.Sound('audio_samples/piano-bb_A#_major.wav'),
    'blue': pygame.mixer.Sound('audio_samples/old-school-hip-hop-pack-vol-2-bright-guitar-stab_C#_minor.wav'),
    'white': pygame.mixer.Sound('audio_samples/Cm_ViolinPizzicato_02_235_SP.wav'),
    'orange': pygame.mixer.Sound('audio_samples/Cm_ViolinStraight_01_235_SP.wav'),
    'yellow': pygame.mixer.Sound('audio_samples/piano-g_G_major.wav')
}

min_size = 150
max_size = 200
previous_parameters = {
    'size': 0,
    'brightness': 0,
    'color': (0, 0, 0)  # RGB color
}

# play audio based on the instrument
def play_audio(instrument):
    if instrument in instrument_sounds:
        instrument_sounds[instrument].play()

# OpenCV STAR detector
star_detector = cv2.FastFeatureDetector_create(threshold=20, nonmaxSuppression=True)
video_capture = cv2.VideoCapture('Cosmic eXploration Hubble eXtreme Deep Field.mp4')

while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    # grayscale for edge detection
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Perform edge detection
    edges = cv2.Canny(gray_frame, threshold1=30, threshold2=100)

    keypoints = star_detector.detect(edges)

    # Filter and process keypoints at the edges
    edge_margin = 15  # Adjustable

    for keypoint in keypoints:
        x, y = map(int, keypoint.pt)
        if (
            x < edge_margin
            or y < edge_margin
            or x > frame.shape[1] - edge_margin
            or y > frame.shape[0] - edge_margin
        ):
            # Calculating the diameter of the star
            size = 2 * keypoint.size

            # Avg. brightness of star region
            star_region = frame[y:y+int(size), x:x+int(size)]
            brightness = np.mean(cv2.cvtColor(star_region, cv2.COLOR_BGR2GRAY))

            # Extracting the RGB color of the star
            color = tuple(np.mean(star_region, axis=(0, 1)).astype(int))


            if color[0] > color[1] and color[0] > color[2]:
                instrument = 'red'
            elif color[0] > color[2] and color[1] > color[2]:
                instrument = 'yellow'
            elif color[2] > color[0] and color[2] > color[1]:
                instrument = 'blue'
            elif color[0] > color[1] and color[0] > color[2]:
                instrument = 'white'
            else:
                instrument= 'orange'


            # Play audio when an object of interest is detected
            play_audio(instrument)

            # Visually highlight the object of interest
            cv2.circle(frame, (x, y), 3, (0, 0, 255), -1)

    # Display the frame
    cv2.imshow('Frame', frame)

    # Exit when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close OpenCV windows
video_capture.release()
cv2.destroyAllWindows()
